##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Install / Import / Load

This code depends on [Lucid](https://github.com/tensorflow/lucid) (our visualization library), and [svelte](https://svelte.technology/) (a web framework). The following cell will install both of them, and dependancies such as TensorFlow. And then import them as appropriate.

In [2]:
#!npm install -g svelte-cli@2.2.0
#!pip install --user scikit-image

import numpy as np
import tensorflow as tf
from skimage.transform import resize
from skimage.io import imsave

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

from top5gen.examples.inception_pretrained import get_top_five

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


# Attribution Code

In [3]:
# This, obviously, loads up the model
model = models.InceptionV1()
model.load_graphdef()

In [4]:
# Here we take in the 1000-dimension vector of potential image labels
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
labels = [line[line.find(" "):].strip().encode() for line in labels_str.decode().split("\n")]
labels = [label.decode().strip().split()[1].replace("_", " ") for label in labels]
labels = ["dummy"] + labels

In [5]:
def raw_class_spatial_attr(img, layer, label, override=None):
  """How much did spatial positions at a given layer effect a output class?"""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    # display results

    
    if label is None: return np.zeros(acts.shape[1:-1])
    
    # Compute gradient
    softmax_result = T("softmax2_pre_activation")
    score = softmax_result[0, labels.index(label)] # The "score" is the softmax result at idx [0, label idx]
    t_grad = tf.gradients([score], [T(layer)])[0]   
    grad = t_grad.eval({T(layer) : acts})
    
    # Linear approximation of effect of spatial position
    return np.sum(acts * grad, -1)[0]

In [6]:
def raw_spatial_spatial_attr(img, layer1, layer2, override=None):
  """Attribution between spatial positions in two different layers."""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts1 = T(layer1).eval()
    acts2 = T(layer2).eval({T(layer1) : acts1})
    
    # Construct gradient tensor
    # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
    n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
    layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
    score = layer2_mags[n_x, n_y]
    t_grad = tf.gradients([score], [T(layer1)])[0]
    
    # Compute attribution backwards from each positin in layer2
    attrs = []
    for i in range(acts2.shape[1]):
      attrs_ = []
      for j in range(acts2.shape[2]):
        grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
        # linear approximation of imapct
        attr = np.sum(acts1 * grad, -1)[0]
        attrs_.append(attr)
      attrs.append(attrs_)
  return np.asarray(attrs)

# Spatial Attribution Interface

In this section, we build the *interface* for interacting with the different kinds of spatial attribution data that we can compute using the above functions. Feel free to skip over this if you aren't interested in that part. The main reason we're including it is so that you can change the interface if you want to.


In [7]:
%%html_define_svelte SpatialWidget

<div class="figure" style="width: 10000px; height: 250px; contain: strict;">
  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint1 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3a {{pct1}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint2 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3b {{pct2}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint3 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4a {{pct3}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint4 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4b {{pct4}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint5 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4c {{pct5}}</div>
  </div>

 <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4d {{pct6}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4e {{pct7}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint8 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5a {{pct8}}</div>
  </div>

    <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint9 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5b {{pct9}}</div>
  </div>
  
</div>


<style>

  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .outer img, .outer svg {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .attr {
    opacity: 0.6;
  }
  .pointer_container {
    z-index: 100;
  }
  .pointer_container rect {
    opacity: 0;
  }
  .pointer_container .selected  {
    opacity: 1;
    fill: none;
    stroke: hsl(24, 100%, 50%);
    stroke-width: 0.1px;
  }
  .label{
    position: absolute;
    left: 0px;
    top: 224px;
    width: 224px;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        hint1: "",
        hint2: "",
        hint3: "",
        hint4: "",
        hint5: "",
        hint6: "",
        hint7: "",
        hint8: "",
        hint9: "",
        pct1: "",
        pct2: "",
        pct3: "",
        pct4: "",
        pct5: "",
        pct6: "",
        pct7: "",
        pct8: "",
        pct9: "",
        spritemap1 : "",
        size1: 1,
        spritemap2 : "",
        size2: 1,
        pos1: undefined,
        pos2: undefined,
        layer1: "",
        layer2: ""
      };
    },
    computed: {
      xs1: (size1) => range(size1),
      ys1: (size1) => range(size1),
      xs2: (size2) => range(size2),
      ys2: (size2) => range(size2)
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_prc4s6qp/SpatialWidget_2d10208f_e913_4a63_a23a_f2adc3fa8b42.html > /tmp/svelte_prc4s6qp/SpatialWidget_2d10208f_e913_4a63_a23a_f2adc3fa8b42.js
b'svelte version 1.64.1\ncompiling ../../../tmp/svelte_prc4s6qp/SpatialWidget_2d10208f_e913_4a63_a23a_f2adc3fa8b42.html...\n(4:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(5:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(11:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(12:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(18:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(19:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(25:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(26:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(32:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(33:4) \xe2\x80\x93

In [8]:
def image_url_grid(grid):
  return [[_image_url(img) for img in line] for line in grid ]

def spatial_spatial_attr(img, layer1, layer2, hint_label_1=None, hint_label_2=None, override=None):
  """This is where the actual saliency maps are gen'd from the "raw" functions above"""
  hint1 = orange_blue(
      raw_class_spatial_attr(img, layer1, hint_label_1, override=override),
      raw_class_spatial_attr(img, layer1, hint_label_2, override=override),
      clip=True
  )
  hint2 = orange_blue(
      raw_class_spatial_attr(img, layer2, hint_label_1, override=override),
      raw_class_spatial_attr(img, layer2, hint_label_2, override=override),
      clip=True
  )

  attrs = raw_spatial_spatial_attr(img, layer1, layer2, override=override)
  attrs = attrs / attrs.max()
  
  lucid_svelte.SpatialWidget({
    "spritemap1": image_url_grid(attrs),
    "spritemap2": image_url_grid(attrs.transpose(2,3,0,1)),
    "size1": attrs.shape[3],
    "layer1": layer1,
    "size2": attrs.shape[0],
    "layer2": layer2,
    "img" : _image_url(img),
    "hint1": _image_url(hint1),
    "hint2": _image_url(hint2)
  })

In [17]:
def orange_blue(a,b,clip=False):
  """This is what makes the orange/blue highlights in the saliency map"""
  
  # Clip *is* called on each run in the given code
  # --This appears to cut down on noise (in purple) in the final output
  if clip:
    a,b = np.maximum(a,0), np.maximum(b,0)
    
  # This is all original to what I was given, I don't know what the constants represent
  arr = np.stack([a, (a + b)/2., b], -1)
  arr /= 1e-2 + np.abs(arr).max()/1.5
  arr += 0.3  # Brightens the image a tad
  i = 0
  j = 0
    
  print(type(arr))
  print(arr.shape)
  print(arr.size)
  print(arr.min(), arr.max(), arr.mean())
  print(arr[0,0])
  return
#   with open("arrdata", 'a+') as f:
#     for row in arr:
#         f.write(str(row))
#         f.write('\n')
  
#   for values in arr:

#     for value in values:

#         for a in value:

#           if a == 0.3: i += 1
#           j += 1
  
  return [arr, i/j]

# Simple Attribution Example

In [18]:
layers = ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']
cur_labels = get_top_five()
for label in cur_labels:
    # Note: Must save the file off each time because it will not re-read raw ndarray
    img = resize(load(label.pop(0)), (224, 224))  # Make all 224x224
    imsave("current.jpeg", img, plugin='pil', format_str='jpeg')
    img = load("current.jpeg")
    for tag in label:
        print(tag)
        saliency_layers = []
        saliency_values = []
        for layer in layers:        
            # Get our saliency layer
            saliency = orange_blue(
              raw_class_spatial_attr(img, layer, tag.split(':')[-1].strip(), override=None),
              raw_class_spatial_attr(img, layer, None, override=None),
              clip=True
            )
            saliency_layers.append(saliency[0])
            saliency_values.append(saliency[1])

            attrs = raw_spatial_spatial_attr(img, layer, "mixed5b", override=None)
            attrs = attrs / attrs.max()

#         lucid_svelte.SpatialWidget({
#         "spritemap1": image_url_grid(attrs),
#         "spritemap2": image_url_grid(attrs.transpose(2,3,0,1)),
#         "size1": attrs.shape[3],
#         "layer1": layer,
#         "size2": attrs.shape[0],
#         "layer2": layer,
#         "img" : _image_url(img),
#         "hint1": _image_url(saliency_layers[0]),
#         "hint2": _image_url(saliency_layers[1]),
#         "hint3": _image_url(saliency_layers[2]),
#         "hint4": _image_url(saliency_layers[3]),
#         "hint5": _image_url(saliency_layers[4]),
#         "hint6": _image_url(saliency_layers[5]),
#         "hint7": _image_url(saliency_layers[6]),
#         "hint8": _image_url(saliency_layers[7]),
#         "hint9": _image_url(saliency_layers[8]),
#         "pct1": 100 - saliency_values[0] * 100,
#         "pct2": 100 - saliency_values[1] * 100,
#         "pct3": 100 - saliency_values[2] * 100,
#         "pct4": 100 - saliency_values[3] * 100,
#         "pct5": 100 - saliency_values[4] * 100,
#         "pct6": 100 - saliency_values[5] * 100,
#         "pct7": 100 - saliency_values[6] * 100,
#         "pct8": 100 - saliency_values[7] * 100,
#         "pct9": 100 - saliency_values[8] * 100
#         })

Load conv1_7x7_s2 weights!
Load conv1_7x7_s2 biases!
Load conv2_3x3_reduce weights!
Load conv2_3x3_reduce biases!
Load conv2_3x3 weights!
Load conv2_3x3 biases!
Load inception_3a_1x1 weights!
Load inception_3a_1x1 biases!
Load inception_3a_3x3_reduce weights!
Load inception_3a_3x3_reduce biases!
Load inception_3a_3x3 weights!
Load inception_3a_3x3 biases!
Load inception_3a_5x5_reduce weights!
Load inception_3a_5x5_reduce biases!
Load inception_3a_5x5 weights!
Load inception_3a_5x5 biases!
Load inception_3a_pool_proj weights!
Load inception_3a_pool_proj biases!
Load inception_3b_1x1 weights!
Load inception_3b_1x1 biases!
Load inception_3b_3x3_reduce weights!
Load inception_3b_3x3_reduce biases!
Load inception_3b_3x3 weights!
Load inception_3b_3x3 biases!
Load inception_3b_5x5_reduce weights!
Load inception_3b_5x5_reduce biases!
Load inception_3b_5x5 weights!
Load inception_3b_5x5 biases!
Load inception_3b_pool_proj weights!
Load inception_3b_pool_proj biases!
Load inception_4a_1x1 weigh

/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


1: probability: 0.89, label: paddle
<class 'numpy.ndarray'>
(28, 28, 3)
2352
0.3 1.7807576433261125 0.3263250403646833
[0.33558258 0.31779129 0.3       ]


TypeError: 'NoneType' object is not subscriptable